<a href="https://colab.research.google.com/github/Nik8x/Deep_writing_generating_text/blob/master/deep_writing-tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import regex as re

import nltk
from nltk.draw.dispersion import dispersion_plot
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.probability import FreqDist
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
nltk.download('averaged_perceptron_tagger')
from nltk.util import ngrams
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

from sklearn.preprocessing import LabelBinarizer

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

import urllib

import os

# Any results you write to the current directory are saved as output.
# from google.colab import drive
# drive.mount('/content/gdrive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Using TensorFlow backend.


In [0]:
# print("The current directory is: ", os.getcwd())
# import os
# os.chdir("/content/gdrive/My Drive/Galvanize Adm/Marcel Proust")
# print("The current directory is: ", os.getcwd())

In [3]:
txt = urllib.request.urlopen('http://www.textfiles.com/stories/3gables.txt').read().decode('utf8')
txt = txt.replace('\n', ' ')[322:]
txt[0:400]

"The Adveniure of the Three Gables\r \r   I don't think that any of my adventures with Mr. Sherlock\r Holmes opened quite so abruptly, or so dramatically, as that\r which I associate with The Three Gables. I had not seen Holmes\r for some days and had no idea of the new channel into which his\r activities had been directed. He was in a chatty mood that\r morning, however, and had just settled me into the "

In [0]:
# Let's do some cleaning

In [0]:
words = txt.split(' ')
words_s = [word.lower() for word in words if word.isalpha()]

In [6]:
len(words_s)

4448

In [0]:
# |# creating characters, words and lists 
# characters = sorted(list(set(txt)))
# words = txt.split(' ')
# sentences = txt.split('.')

In [0]:
# # let's remove stop words from words
# stop_words = set(stopwords.words('english'))
# words_s = [i for i in words if not i in stop_words]
# words_s = [word.lower() for word in words_s if word.isalpha()]

In [0]:
# fdist = FreqDist(words_s) # checking most frequent words in whole document
# plt.figure(figsize= (12,2))
# plt.bar(pd.DataFrame(fdist.most_common()[0:20])[0], pd.DataFrame(fdist.most_common()[0:20])[1])
# plt.xticks(rotation=90)
# plt.show()

In [0]:
# # lemmatizing
# lemmatizer = WordNetLemmatizer()
# words_s_l = pd.Series(words_s).apply(lambda x: lemmatizer.lemmatize(x))

In [0]:
# words_s_l_p = pd.DataFrame(nltk.pos_tag(words_s_l), columns = ['words', 'pos'])

In [0]:
sentences = ' '.join(words_s)

In [13]:
# Let's look at overall sentiment of the book
scores = SentimentIntensityAnalyzer().polarity_scores(sentences)
print(scores)

{'neg': 0.074, 'neu': 0.801, 'pos': 0.125, 'compound': 0.9998}


In [0]:
# saving sentences as bigrams

In [0]:
def n_gram(x):
  tokens = [token for token in x.split(" ") if token != ""]
  output = list(ngrams(tokens, 3))
  return output

In [16]:
df = pd.DataFrame(pd.Series(sentences).apply(lambda x: n_gram(x))[0])

# make a dataframe with 2 words as independent and just next third word as target column
df = pd.concat([df[0] + ' ' + df[1], df[2]], axis = 1)
df.head()

,0,2
0,the adveniure,of
1,adveniure of,the
2,of the,three
3,the three,i
4,three i,think


In [17]:
df.shape

(4446, 2)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfid = TfidfVectorizer()
X = tfid.fit_transform(df[0]).toarray()

In [19]:
X.shape

(4446, 1046)

In [20]:
encoder = LabelBinarizer()
y = encoder.fit_transform(df[2])
y.shape

(4446, 1048)

In [0]:
# LSTMs accept input in the form of (number_of_sequences, length_of_sequence, number_of_features)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [22]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(100, input_shape=(X.shape[1], X.shape[2]), return_sequences = True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(100))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [23]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.92.184.58:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 17276491466978196900)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3382443476736315270)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6749064115886791912)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1

In [24]:
tpu_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_input (InputLayer)      (None, 1046, 1)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 1046, 100)         40800     
_________________________________________________________________
dropout (Dropout)            (None, 1046, 100)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1048)              105848    
Total params: 227,048
Trainable params: 227,048
Non-trainable params: 0
_________________________________________________________________


In [29]:
tpu_model.fit(X, y, epochs = 50, batch_size = 100)

Epoch 1/50
4446/4446 [==============================] - 7s 2ms/sample - loss: 5.7419 - acc: 0.0446
Epoch 2/50
4446/4446 [==============================] - 7s 2ms/sample - loss: 5.6961 - acc: 0.0509
Epoch 3/50
4446/4446 [==============================] - 7s 2ms/sample - loss: 5.6964 - acc: 0.0518
Epoch 4/50
4446/4446 [==============================] - 7s 2ms/sample - loss: 5.6798 - acc: 0.0523
Epoch 5/50
4446/4446 [==============================] - 7s 2ms/sample - loss: 5.6826 - acc: 0.0514
Epoch 6/50
4446/4446 [==============================] - 7s 2ms/sample - loss: 5.6688 - acc: 0.0497
Epoch 7/50
4446/4446 [==============================] - 7s 2ms/sample - loss: 5.6727 - acc: 0.0516
Epoch 8/50
4446/4446 [==============================] - 7s 2ms/sample - loss: 5.6657 - acc: 0.0532
Epoch 9/50
4446/4446 [==============================] - 7s 2ms/sample - loss: 5.6652 - acc: 0.0546
Epoch 10/50
4446/4446 [==============================] - 7s 2ms/sample - loss: 5.6599 - acc: 0.0537
Epoch 11/

In [30]:
cpu_model = tpu_model.sync_to_cpu()

INFO:tensorflow:Copying TPU weights to the CPU
INFO:tensorflow:TPU -> CPU lr: 0.0010000000474974513
INFO:tensorflow:TPU -> CPU beta_1: 0.8999999761581421
INFO:tensorflow:TPU -> CPU beta_2: 0.9990000128746033
INFO:tensorflow:TPU -> CPU decay: 0.0
INFO:tensorflow:TPU -> CPU epsilon: 1e-07
INFO:tensorflow:TPU -> CPU amsgrad: False


In [0]:
y_pred = cpu_model.predict(X[99:150])

In [32]:
(encoder.inverse_transform(y_pred)).tolist()

['the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the']

In [33]:
%%time
# Evaluate the model on valid set
score = cpu_model.evaluate(X, y, verbose = 0)

# Print test accuracy
print('\n', 'Valid accuracy:', score[1])


 Valid accuracy: 0.053081423
CPU times: user 251 ms, sys: 41.6 ms, total: 293 ms
Wall time: 49.7 s
